In [1]:
using Distributed
# @everywhere used for the implementation of the modules across multiple workers
# Without it, the code does not run

# Add the modules to the ´read´ path of the Julia
@everywhere push!(LOAD_PATH, "/scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/")


# Custom modules
@everywhere using Model: HapticNet, VisualNet, train_epoch!, save_as_jld2
@everywhere using TUM69: load_accel_data, load_image_data
@everywhere using Preprocess: process_accel_signal, process_image, augment_image
@everywhere using Utils: kfold, notify!, a_type
# Ready-to-use modules
using JLD2, Random
using CUDA
using Augmentor: FlipX, FlipY

In [22]:
# Set path
path = CUDA.functional() ? "/userfiles/vaydingul20/data/new" : "data/new" # path of the main data
DATA_PATH = isdir(path) && path
# Set pretrained network path
alexnet_PATH = "/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

"/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

In [23]:
# Load image data
X_train, y_train, _, _, material_dict = @time load_image_data(DATA_PATH; mode="baseline");

Train image data is being loaded!
Test image data is being loaded!
 44.510236 seconds (31.33 k allocations: 9.244 GiB, 0.31% gc time)


(Array{ColorTypes.RGB{FixedPointNumbers.Normed{UInt8,8}},2}[[RGB{N0f8}(0.733,0.655,0.408) RGB{N0f8}(0.733,0.655,0.408) … RGB{N0f8}(0.863,0.824,0.541) RGB{N0f8}(0.863,0.824,0.541); RGB{N0f8}(0.737,0.659,0.412) RGB{N0f8}(0.725,0.647,0.4) … RGB{N0f8}(0.859,0.82,0.537) RGB{N0f8}(0.851,0.812,0.529); … ; RGB{N0f8}(0.545,0.522,0.294) RGB{N0f8}(0.545,0.518,0.302) … RGB{N0f8}(0.808,0.694,0.576) RGB{N0f8}(0.812,0.69,0.576); RGB{N0f8}(0.549,0.529,0.302) RGB{N0f8}(0.553,0.533,0.306) … RGB{N0f8}(0.82,0.698,0.584) RGB{N0f8}(0.824,0.702,0.588)], [RGB{N0f8}(0.525,0.482,0.475) RGB{N0f8}(0.529,0.486,0.478) … RGB{N0f8}(0.682,0.643,0.545) RGB{N0f8}(0.655,0.616,0.518); RGB{N0f8}(0.525,0.482,0.475) RGB{N0f8}(0.518,0.475,0.467) … RGB{N0f8}(0.69,0.651,0.553) RGB{N0f8}(0.675,0.635,0.537); … ; RGB{N0f8}(0.384,0.337,0.29) RGB{N0f8}(0.361,0.314,0.267) … RGB{N0f8}(0.494,0.459,0.424) RGB{N0f8}(0.49,0.455,0.42); RGB{N0f8}(0.392,0.345,0.298) RGB{N0f8}(0.373,0.325,0.278) … RGB{N0f8}(0.502,0.467,0.431) RGB{N0f8}(0.506,

In [24]:
# Augmentation pipeline
p1 = FlipX()
p2 = FlipY()
p3 = FlipX() |> FlipY()
X_train, y_train = @time augment_image(X_train, y_train, p1, p2, p3);

 10.211942 seconds (433 allocations: 4.621 GiB, 2.18% gc time)


In [25]:
# Apply preprocessing on the images
# Here is where the code explodes
X_train, y_train = @time process_image(X_train, y_train);

109.515163 seconds (134.87 k allocations: 28.019 GiB, 16.79% gc time)


In [ ]:
# Seperate into 3 folds for training
kf = kfold(X_train, y_train; fold=3, minibatch_size = 10, atype=a_type(Float32))
results = []

In [21]:
for (ix, (dtrn, dtst)) in enumerate(kf.folds[1:1])
    

    #Reset the model
    vn = VisualNet(alexnet_PATH; atype=a_type(Float32))

    for k in 1:1
        # Training routine
        res = train_epoch!(vn, dtrn, dtst; progress_bar=true, fig=false, info=true, epoch=100)
        # Save model 
        # Add results to the ´results´vector
        push!(results, res)

    end
    save_as_jld2(vn, "vn-$ix.jld2")

end

┣████████████████████┫ [100.00%, 4800/4800, 03:37/03:37, 22.10i/s] 


TRAINING PARAMETERS


Activation Function = relu
Optimizer Type = adam
Learning Rate = 0.0001
LAYERS:


Model.Conv ==> W = (11, 11, 3, 48)   b = (1, 1, 48, 1)
Model.Conv ==> W = (5, 5, 48, 256)   b = (1, 1, 256, 1)
Model.Conv ==> W = (3, 3, 256, 192)   b = (1, 1, 192, 1)
Model.Conv ==> W = (3, 3, 192, 192)   b = (1, 1, 192, 1)
Model.Conv ==> W = (3, 3, 192, 256)   b = (1, 1, 256, 1)
Model.Conv ==> W = (6, 6, 256, 300)   b = (1, 1, 300, 1)
Model.Conv ==> W = (1, 1, 300, 250)   b = (1, 1, 250, 1)
Model.Conv ==> W = (1, 1, 250, 69)   b = (1, 1, 69, 1)
In this network configuration,
there are total 4399749 parameters.
Final Loss
Train Loss = 0.00023503655
Test Loss = 0.0010540287


Final Misclassification Error
Train Misclassification Error = 0.0
Test Misclassification Error = 0.0


Test Accuracy = 1.0


In [11]:
# Save accumulated results
JLD2.@save "results_vn.jld2" results = results